In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.226.1:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1738932140359)
SparkSession available as 'spark'


import org.apache.spark


In [2]:
val path_to_bigDatasets = "../../../../datasets/big/"
val path_to_datasets = "../../../../datasets/"

val path_Fullml_posts = path_to_bigDatasets + "the-reddit-covid-dataset-posts.csv"
val path_Fullml_comments = path_to_bigDatasets + "the-reddit-covid-dataset-comments.csv"

val path_sample_posts = path_to_datasets + "postsSample.csv"
val path_sample_comments = path_to_datasets + "commentsSample.csv"

path_to_bigDatasets: String = ../../../../datasets/big/
path_to_datasets: String = ../../../../datasets/
path_Fullml_posts: String = ../../../../datasets/big/the-reddit-covid-dataset-posts.csv
path_Fullml_comments: String = ../../../../datasets/big/the-reddit-covid-dataset-comments.csv
path_sample_posts: String = ../../../../datasets/postsSample.csv
path_sample_comments: String = ../../../../datasets/commentsSample.csv


In [8]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object CovidConversationsParser {

    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val quotes = "\""

    /** Convert from timestamp (String) to year (Int) */
    def yearFromTimestamp(timestamp: String): Int = {
        val cal = Calendar.getInstance()
        cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
        cal.get(Calendar.YEAR)
    }

    /** Function to parse reddit posts
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score. none in case of input errors
    */
    def parseRedditPost(line: String): Option[(String, String, String, Boolean, Int, String, String, String, String, String, Int)] = {
        try {
            val input = line.split(commaRegex)
            if(input.size != 12) {
                return Some(("", "", "", false, -1, "", "", "", "", "", -1))
            }   
            val url = if(input(8).trim.nonEmpty) input(8).trim else "None"
            val selftext = if(input(9).trim.nonEmpty) input(9) else "None"
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, input(7).trim, url, selftext, input(10).trim , input(11).trim.toInt))   
        } catch {
            case _: Exception => None
        }
    }

    /** Function to parse reddit comments
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score. none in case of input errors
    */
    def parseRedditComment(line: String): Option[(String, String, String, Boolean, Int, String, String, Double, Int)] = {
        try {
            val input = line.split(commaRegex)
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean,yearFromTimestamp(input(5)), input(6).trim, input(7).trim, input(8).trim.toDouble, input(9).trim.toInt))
        } catch {
            case _: Exception => None
        }
    }
}

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
defined object CovidConversationsParser


In [13]:
val data = Array(1, 2, 3, 4, 5)
val rddData = sc.parallelize(data)
rddData.filter(x => x < 4).collect()

data: Array[Int] = Array(1, 2, 3, 4, 5)
rddData: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[16] at parallelize at <console>:35
res7: Array[Int] = Array(1, 2, 3)


In [110]:
val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
val value = "post,qfsoln,2qs1t,newyorkcity,false,1635202606,https://old.reddit.com/r/newyorkcity/comments/qfsoln/live_now_78pm_pix11_holds_their_2021_nyc_mayoral/,pix11.com,https://pix11.com/news/politics/new-york-elections/pix-on-politics-adams-sliwa-to-face-off-in-nyc-mayoral-forum-monday/,,\"Live Now (7-8PM): Pix11 holds their 2021 NYC mayoral forum, with frontrunner candidates Eric Adams (D), and Curtis Sliwa (R) answering questions on city issues such as COVID-19, vaccine mandates, schools, crime, youth violence and much more. (10/25/2021)\",1"
value.split(commaRegex).size

commaRegex: String = ,(?=(?:[^"]*"[^"]*")*[^"]*$)
value: String = post,qfsoln,2qs1t,newyorkcity,false,1635202606,https://old.reddit.com/r/newyorkcity/comments/qfsoln/live_now_78pm_pix11_holds_their_2021_nyc_mayoral/,pix11.com,https://pix11.com/news/politics/new-york-elections/pix-on-politics-adams-sliwa-to-face-off-in-nyc-mayoral-forum-monday/,,"Live Now (7-8PM): Pix11 holds their 2021 NYC mayoral forum, with frontrunner candidates Eric Adams (D), and Curtis Sliwa (R) answering questions on city issues such as COVID-19, vaccine mandates, schools, crime, youth violence and much more. (10/25/2021)",1
res84: Int = 12


In [ ]:
//per recuperare dei samples dei files
//sc.textFile(path_Fullml_posts).sample(false, 0.05).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")
sc.textFile(path_Fullml_comments).sample(false, 0.05).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")

In [17]:
/* se osserva qui prof, quando prendo semplicemente il primo elemento da quelli indicati, sembra perfettamente normale*/
val rddPosts = sc.textFile(path_sample_posts)
rddPosts.first()

rddPosts: org.apache.spark.rdd.RDD[String] = ../../../../datasets/postsSample.csv MapPartitionsRDD[29] at textFile at <console>:34
res11: String = "post,qftrls,4z7iqj,auscovid19,false,1635206035,https://old.reddit.com/r/AusCOVID19/comments/qftrls/queensland_records_two_new_community_covid19_cases/,abc.net.au,https://www.abc.net.au/news/2021-10-26/qld-coronavirus-covid-cases-home-quarantine-community/100551264,,Queensland records two new community COVID-19 cases,1"


In [16]:
/*se faccio qui la filter, ottengo però questo "posted at 2020-05-11 17:15 CET by Rafael Antonio Pineda" e non ho la minima idea del perchè????*/
val rddPosts = sc.textFile(path_sample_posts).filter(x => x.startsWith("post"))
rddPosts.first()

rddPosts: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[27] at filter at <console>:34
res10: String = posted at 2020-05-11 17:15 CET by Rafael Antonio Pineda


In [15]:
/*getting RDDs of Comments and Posts*/
/* qui prof ho provato a cambiare il parser per fare sì che creasse delle tuple vuote se dopo lo split, non ottenevo 12 valori, eppure l unica cosa mi genera è una serie di queste tuple vuote, perche???*/
val rddPosts = sc.textFile(path_sample_posts).flatMap(CovidConversationsParser.parseRedditPost)
rddPosts.collect()

commaRegex: String = ,(?=(?:[^"]*"[^"]*")*[^"]*$)
rddPosts: org.apache.spark.rdd.RDD[(String, String, String, Boolean, Int, String, String, String, String, String, Int)] = MapPartitionsRDD[24] at flatMap at <console>:37
res9: Array[(String, String, String, Boolean, Int, String, String, String, String, String, Int)] = Array(("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("","","",false,-1,"","","","","",-1), ("...


In [ ]:
//PART 1: Aggregate on temporal dimension and obtain percentage of posts classified as NSFW
//Posts are :(id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score)
val percentageNSFWPosts = rddPosts.map(x => (x._5, (x._1, x._3, if (x._4) 1 else 0)))  // (created_utc, nsfw flag)
.reduceByKey(_ + _)  // count NSFW posts per timestamp
.mapValues(nsfwCount => {
    val totalPostsAtTime = rddPosts.filter(y => y._5 == y._5).count()
    (nsfwCount.toDouble / totalPostsAtTime) * 100
})

//.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
//.collect()

In [ ]:
val percentageNSFWPosts = rddPosts.map(x => (x._5, (x._1, x._3, if (x._4) 1 else 0)))  // (timestamp, (id, subreddit_name, nsfw flag)).aggregateByKey(
    (0, 0)  // (nsfwCount, totalCount)
)(
    // Accumulator for each partition
    (acc, value) => (acc._1 + value._3, acc._2 + 1),
    // Combiner for merging partitions
    (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
).map { case (timestamp, (nsfwCount, totalCount)) =>
    val percentage = (nsfwCount.toDouble / totalCount) * 100
    (timestamp, rddPosts.filter(_._5 == timestamp).first()._1,  // id
    rddPosts.filter(_._5 == timestamp).first()._3,  // subreddit_name
    percentage)
}

In [ ]:
//PART 2: Aggregate on temporal dimension and obtain average sentiment in comments and percentage of comments classified as NSFW
//Comments are: (id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score)
val avgSentimentWithNSFWPost = rddComments.map(x => (x._5, (if (x._4) 1 else 0, x._8)))  // (created_utc, (nsfw flag, sentiment))
.reduceByKey((a, b) => (a._1 + b._1, a._2 + b._2))  // aggregate NSFW count and sentiment
.mapValues(avgSentimentAndNSFWCount => {
    val totalCommentsAtTime = rddComments.filter(y => y._5 == y._5).count()
    val nsfwPercentage = (avgSentimentAndNSFWCount._1.toDouble / totalCommentsAtTime) * 100
    val avgSentiment = avgSentimentAndNSFWCount._2 / totalCommentsAtTime
    (nsfwPercentage, avgSentiment)
})

In [ ]:
// PART 3: Join on Temporal Dimension
val finalResult = percentageNSFWPosts.join(avgSentimentWithNSFWPost)